# Setting the timezone for HPE iLO systems

<div class="alert  alert-block  alert-info" style="border-radius: 20px;">    
    <ul>
        <li><p>This Jupyter Notebook demonstrates an example of setting the timezone for HPE iLO (Integrated Lights-Out) systems. It utilizes the HPE iLOrest library to interact with the iLO API and retrieve the necessary information.</p></li>
        <li>As you are here let's get it done easily.</li>
        <li>To perform this you need to have <a href = "https://libraries.io/pypi/python-ilorest-library#id3" style="none">python-ilorest-library</a> installed in your machine.</li>
        <li>Please keep your username and password handy</li>
        <li>Do execute each successive cell once to avoid any errors!</li>
    </ul>    
</div>

<div class="alert alert-block alert-success" style="border: 1px solid #ccc; border-radius: 10px; padding: 10px;">
    <div style = "color:green; font-size:40px;">1.</div><br>
Below we are importing all the necessary modules from the ilorest library tp set up the ilo NTP server!<br>
  
</div>

In [1]:
import sys
import json
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError

from get_resource_directory import get_resource_directory

<div class = "alert alert-block alert-success" style="border: 1px solid #ccc; border-radius: 10px; padding: 10px;">
    <div style = "color:green; font-size:40px;">2.</div><br>
    <p>The function set_timezone is defined below. It provides a convenient way to set the timezone for an HPE iLO system using the Redfish API, utilizing the resource directory or direct URI retrieval methods to locate the relevant DateTime service and perform the timezone update.</p>
<br> Here's a summary of the function:<br>

1. <b>Resource Directory Retrieval</b>: The function retrieves the resource directory information using the _redfishobj Redfish client object. If the resource directory is not available or the DISABLE_RESOURCE_DIR flag is set, it retrieves the URI for the DateTime service from the server's Managers and Members resources.

2. <b>Finding the Relevant URI</b>: If the resource directory is available, the function searches for the relevant URI for the DateTime service within the resource instances. It looks for an instance with the '#HpeiLODateTime.' substring in its @odata.type.

3. <b>Updating the Timezone</b>: After obtaining the DateTime service URI, the function sends a GET request to retrieve the current data for the service. If the retrieved data contains the 'TimeZone' property, it constructs a JSON payload to update the timezone to the specified value. A PATCH request is then sent to the DateTime service URI with the timezone payload to modify the timezone.

4. <b>Handling Responses</b>: The function handles different response scenarios. If the response status code is 400, indicating a client-side error, it attempts to print the extended error information and the available timezone list. For response status codes other than 200 or 201, indicating an HTTP error, an appropriate error message is printed. If the response status is 200, indicating a successful update, the function prints a success message along with the updated timezone information.
</div>

In [1]:
def set_timezone(_redfishobj, timezone):

    date_time_uri = None

    resource_instances = get_resource_directory(_redfishobj)
    if DISABLE_RESOURCE_DIR or not resource_instances:
        #if we do not have a resource directory or want to force it's non use to find the
        #relevant URI
        managers_uri = _redfishobj.root.obj['Managers']['@odata.id']
        managers_response = _redfishobj.get(managers_uri)
        managers_members_uri = next(iter(managers_response.obj['Members']))['@odata.id']
        managers_members_response = _redfishobj.get(managers_members_uri)
        date_time_uri = managers_members_response.obj.Oem.Hpe.Links['DateTimeService']['@odata.id']
    else:
        #Use Resource directory to find the relevant URI
        for instance in resource_instances:
            if '#HpeiLODateTime.' in instance['@odata.type']:
                date_time_uri = instance['@odata.id']

    if date_time_uri:
        data = _redfishobj.get(date_time_uri)
        if data.dict.get('TimeZone'):
            _data = {'TimeZone': {"Name": timezone}}
            resp = _redfishobj.patch(date_time_uri, _data)
        else:
            raise Exception("\'TimeZone\' property is not available/modifyable.\n")
        #If iLO responds with soemthing outside of 200 or 201 then lets check the iLO extended info
        #error message to see what went wrong
        if resp.status == 400:
            try:
                print(json.dumps(resp.obj['error']['@Message.ExtendedInfo'], indent=4, \
                                                                                sort_keys=True))
                print(json.dumps(data.obj.get('TimeZoneList'), indent=4, sort_keys=True))
                sys.stderr.write("Check the TimeZone value...\n")
            except Exception as excp:
                sys.stderr.write("A response error occurred, unable to access iLO Extended " \
                                 "Message Info...")
        elif resp.status != 200:
            sys.stderr.write("An http response of \'%s\' was returned.\n" % resp.status)
        else:
            print("Success!\n")
            print(json.dumps(resp.dict, indent=4, sort_keys=True))
            ntp_data = _redfishobj.get(date_time_uri).dict.get('TimeZone')
            print("Printing updated NTP Servers:\n")
            print(json.dumps(ntp_data, indent=4, sort_keys=True))

<div class = "alert alert-block alert-success" style="border: 1px solid #ccc; border-radius: 10px; padding: 10px;">
<div style = "color:green; font-size:40px;">3.</div><br>
    This is Main function overview.<br>
    <i>Let me give you a walk through of it!</i>
    <br><br>
    The main section of the script proceeds to create a Redfish client object using the specified system URL, login account, and password. This client object allows communication with the remote system via Redfish APIs. After successfully creating the client object, it logs in to the system to establish an authenticated session.
<br><br>
If any server connectivity issues occur or the server does not support RedFish, an error message is displayed, and the script exits. Otherwise, the set_timezone function is called, passing the Redfish client object and the desired timezone value as parameters. This function handles the task of setting the timezone on the system using the Redfish API.
<br>
Upon completion, the script logs out from the system, terminating the authenticated session.

 </div>

<div class="alert alert-block alert-warning">
<b> When running on the server locally : </b>  use the following values <br>
    SYSTEM_URL = None <br>
    LOGIN_ACCOUNT = None <br>
    LOGIN_PASSWORD = None <br>
    LOGFILE = data.ahs<br>
    <b> When running remotely </b> connect using the secured (https://) address,account name, and password to send https requests <br>
    SYSTEM_URL acceptable examples: <br>
    "https://10.0.0.100" <br>
    "https://ilo.hostname"
</div>

In [ ]:
if __name__ == "__main__":
    
    SYSTEM_URL = "https://10.0.0.100"
    LOGIN_ACCOUNT = "admin"
    LOGIN_PASSWORD = "password"

    #timezone property (review the list of timezones ('TimeZoneList') within the DateTime URI.
    TIMEZONE = "US/Central"
    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = True

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, \
                                                                            password=LOGIN_PASSWORD)
        # Login with the Redfish client
        REDFISHOBJ.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    set_timezone(REDFISHOBJ, TIMEZONE)
    REDFISHOBJ.logout()

<div class = "alert alert-block alert-success" style="border: 1px solid #ccc; border-radius: 10px; padding: 10px;">In case you need help 🔗 links to relevant resources :
    <br>
    1.Python-ilorest-library:<a href = "https://github.com/HewlettPackard/python-ilorest-library">LINK</a><br>
    2.Hpe ilorest-api explorer:<a href = "https://ilorestfulapiexplorer.ext.hpe.com/">LINK</a><br>
    3.Rest api: <a href = "https://restfulapi.net/">LINK</a><br>
    
</div>